In [9]:
import sys
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO

# Add the src directory to the Python path
notebook_dir = os.path.abspath('')
project_root = os.path.dirname(notebook_dir)
src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)

# Verify the directories
print(f"Notebook Directory: {notebook_dir}")
print(f"Project Root: {project_root}")
print(f"Src Directory: {src_dir}")
print(f"Sys Path: {sys.path}")

# Import the style_transfer function
from models.style_transfer import style_transfer

# Define the paths to the content image and the style image
CONTENT_IMG_PATH = os.path.join(src_dir, 'data', 'raw', 'images', 'content', 'background_medium.jpg')
STYLE_IMG_URL = "https://uploads5.wikiart.org/00129/images/katsushika-hokusai/the-great-wave-off-kanagawa.jpg"

# Function to display images
def display_images(content_path, style_url, *images_and_titles):
    content_img = Image.open(content_path)
    response = requests.get(style_url)
    style_img = Image.open(BytesIO(response.content)).convert('RGB')

    fig, axes = plt.subplots(1, len(images_and_titles) + 2, figsize=(20, 5))
    axes[0].imshow(content_img)
    axes[0].set_title('Content Image')
    axes[0].axis('off')

    axes[1].imshow(style_img)
    axes[1].set_title('Style Image')
    axes[1].axis('off')

    for i, (image, title) in enumerate(images_and_titles):
        axes[i + 2].imshow(image)
        axes[i + 2].set_title(title)
        axes[i + 2].axis('off')

    plt.show()

# Test the style_transfer function with different configurations
def test_style_transfer(content_img_path, style_img_url, **kwargs):
    output_img = style_transfer(content_img_path, style_img_url, **kwargs)
    return output_img

# Default parameters for testing
default_params = {
    "content_weight": 1e5,
    "style_weight": 1e10,
    "num_steps": 300,
    "content_layers": ['conv_4'],
    "style_layers": ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5'],
    "optimizer_type": 'LBFGS'
}


Notebook Directory: /home/gritchou/code/gritchou/text2play/notebooks
Project Root: /home/gritchou/code/gritchou/text2play
Src Directory: /home/gritchou/code/gritchou/text2play/src
Sys Path: ['/home/gritchou/code/gritchou/04-Decision-Science/01-Project-Setup/data-context-and-setup', '/home/gritchou/code/gritchou/text2play/notebooks', '/home/gritchou/.pyenv/versions/3.10.6/lib/python310.zip', '/home/gritchou/.pyenv/versions/3.10.6/lib/python3.10', '/home/gritchou/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload', '', '/home/gritchou/.pyenv/versions/3.10.6/envs/text2play/lib/python3.10/site-packages', '/home/gritchou/code/gritchou/text2play/src', '/home/gritchou/code/gritchou/text2play/src', '/home/gritchou/code/gritchou/text2play/src', '/home/gritchou/code/gritchou/text2play/src', '/home/gritchou/code/gritchou/text2play/src', '/home/gritchou/code/gritchou/text2play/src', '/home/gritchou/code/gritchou/text2play/src', '/home/gritchou/code/gritchou/text2play/src', '/home/gritchou/code/grit

ModuleNotFoundError: No module named 'models.style_transfer'

In [ ]:
# Test 1: Compare different optimizers
output_lbfgs = test_style_transfer(CONTENT_IMG_PATH, STYLE_IMG_URL, **default_params)
output_adam = test_style_transfer(CONTENT_IMG_PATH, STYLE_IMG_URL, **{**default_params, "optimizer_type": "Adam"})
display_images(CONTENT_IMG_PATH, STYLE_IMG_URL, (output_lbfgs, "LBFGS Optimizer"), (output_adam, "Adam Optimizer"))

In [ ]:
# Test 2: Compare different weights
output_low_content_weight = test_style_transfer(CONTENT_IMG_PATH, STYLE_IMG_URL, **{**default_params, "content_weight": 1})
output_high_content_weight = test_style_transfer(CONTENT_IMG_PATH, STYLE_IMG_URL, **{**default_params, "content_weight": 1e5})
output_low_style_weight = test_style_transfer(CONTENT_IMG_PATH, STYLE_IMG_URL, **{**default_params, "style_weight": 1e3})
output_high_style_weight = test_style_transfer(CONTENT_IMG_PATH, STYLE_IMG_URL, **{**default_params, "style_weight": 1e10})
display_images(
    CONTENT_IMG_PATH, STYLE_IMG_URL,
    (output_low_content_weight, "Low Content Weight"),
    (output_high_content_weight, "High Content Weight"),
    (output_low_style_weight, "Low Style Weight"),
    (output_high_style_weight, "High Style Weight")
)

In [ ]:
# Test 3: Compare different layers
output_few_layers = test_style_transfer(CONTENT_IMG_PATH, STYLE_IMG_URL, **{**default_params, "content_layers": ['conv_4'], "style_layers": ['conv_1', 'conv_2']})
output_many_layers = test_style_transfer(CONTENT_IMG_PATH, STYLE_IMG_URL, **{**default_params, "content_layers": ['conv_4'], "style_layers": ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']})
display_images(
    CONTENT_IMG_PATH, STYLE_IMG_URL,
    (output_few_layers, "Few Style Layers"),
    (output_many_layers, "Many Style Layers")
)